# Importation des packages 

Nous chargons les modules necessaires pour notre analyse. 

In [168]:
#Importation des packages

import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from kmodes.kmodes import KModes
import os 

In [169]:
# Definir le repertoire de travail 
pwd = "C:\\Users\\mouss\\OneDrive\\Documents\\TotalEnergies" 
os.chdir(pwd) 
 
# Afficher le nouveau repertoire de travail 
print(os.getcwd()) 

C:\Users\mouss\OneDrive\Documents\TotalEnergies


# Données
Nous chargeons les quatre jeux de données pour l'analyse en commencant par les données des profils clients. Cette première table contient les données de **177 624 clients** (observations) et de **14 facteurs** caracterisant ces derniers (variables). 

In [170]:
# Importation des données

customer = pd.read_csv("table_customer_profile.csv",sep = ";"
                       #,index_col = "EP_CODE"
                      , encoding= 'latin-1')

In [171]:
#customer.info()

# Echantillon
Ci dessous un échantilon de cette table.

In [172]:
customer.head()

,EP_CODE,EP_STATUS,EP_TYPE,EP_NACE_CODE,EP_NACE_NAME,EP_DATE_ACTIVATION,EP_DATE_LAST_TRAN,EP_POSTCODE,EP_CITY,EP_COUNTRY,EP_SALES_DEPARTMENT,EP_SALES_TEAM,EP_FLEET_CATEGORY,EP_FLEET_TYPE
0,87285956,Activé,Standard,4321,Installation électrique,22/11/2017,11/08/2021,67400,ILLKIRCH GRAFFENSTADEN,FRANCE,Ventes Locales,Ventes Locales : Région Est,Véhicules Légers,VL_MIXTE
1,87205901,Activé,Standard,4332,Travaux de menuiserie,13/01/2014,31/07/2021,89000,AUXERRE,FRANCE,Ventes Locales,Ventes Locales : Région Est,Véhicules Légers,VL_MIXTE
2,87068998,Activé,Standard,NaN,Fabrication de structures métalliques et de pa...,NaN,11/08/2021,14125,MONDEVILLE CEDEX,FRANCE,Ventes Nationales,Ventes Nationales : Région Nord-Est,Véhicules Légers,VL_MIXTE
3,87046955,Activé,Standard,4120,Construction de bâtiments résidentiels et non ...,25/06/2002,10/08/2021,51208,EPERNAY CEDEX,FRANCE,Ventes Nationales,Ventes Nationales : Région Nord-Est,Véhicules Légers,VL_MIXTE
4,87317118,Activé,Standard,220,Exploitation forestière,08/04/2019,11/08/2021,27000,EVREUX,FRANCE,Ventes Locales,Ventes Locales : Région Nord,Véhicules Légers,VL_MIXTE


# Nettoyage et preparation de données

## Types des variables

Dans cette partie, il s'agit de regarder de plus près le type de chaque variable et decider s'il est pertinent de les convertir vers d'autres types. Pour ce faire, analysons la sortie suivante : 

In [173]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177624 entries, 0 to 177623
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   EP_CODE              177624 non-null  int64 
 1   EP_STATUS            177624 non-null  object
 2   EP_TYPE              177624 non-null  object
 3   EP_NACE_CODE         170176 non-null  object
 4   EP_NACE_NAME         165694 non-null  object
 5   EP_DATE_ACTIVATION   170176 non-null  object
 6   EP_DATE_LAST_TRAN    176746 non-null  object
 7   EP_POSTCODE          177624 non-null  object
 8   EP_CITY              177624 non-null  object
 9   EP_COUNTRY           177624 non-null  object
 10  EP_SALES_DEPARTMENT  177624 non-null  object
 11  EP_SALES_TEAM        177624 non-null  object
 12  EP_FLEET_CATEGORY    177624 non-null  object
 13  EP_FLEET_TYPE        177624 non-null  object
dtypes: int64(1), object(13)
memory usage: 19.0+ MB


On remaruqe que toute les variables sont de type *object* (correspondant à string) sauf EP_CODE qui est de type entier. Le type *object* ne pose aucun problème pour les variables qui sont de nature categorielles. C'est le cas pour toutes les variables dans cette table sauf pour EP_DATE_ACTIVATION, EP_DATE_LAST_TRAN et bien sur EP_CODE. En realité, les deux premières susmentionnées sont des dates, donc pour une manipulation correcte, facile et efficace de celles-ci, nous les avons convertis en *datetime*.  

In [174]:
# Convertir les variables EP_DATE_ACIVATION et EP_DATE_LAST_TRAN en type date    ,"EP_DATE_LAST_TRAN"

customer["EP_DATE_ACTIVATION"] = pd.to_datetime(customer.EP_DATE_ACTIVATION,dayfirst=True)
customer["EP_DATE_LAST_TRAN"] = pd.to_datetime(customer.EP_DATE_LAST_TRAN,dayfirst=True)

customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177624 entries, 0 to 177623
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   EP_CODE              177624 non-null  int64         
 1   EP_STATUS            177624 non-null  object        
 2   EP_TYPE              177624 non-null  object        
 3   EP_NACE_CODE         170176 non-null  object        
 4   EP_NACE_NAME         165694 non-null  object        
 5   EP_DATE_ACTIVATION   170176 non-null  datetime64[ns]
 6   EP_DATE_LAST_TRAN    176746 non-null  datetime64[ns]
 7   EP_POSTCODE          177624 non-null  object        
 8   EP_CITY              177624 non-null  object        
 9   EP_COUNTRY           177624 non-null  object        
 10  EP_SALES_DEPARTMENT  177624 non-null  object        
 11  EP_SALES_TEAM        177624 non-null  object        
 12  EP_FLEET_CATEGORY    177624 non-null  object        
 13  EP_FLEET_TYPE 

In [175]:
customer.tail()

,EP_CODE,EP_STATUS,EP_TYPE,EP_NACE_CODE,EP_NACE_NAME,EP_DATE_ACTIVATION,EP_DATE_LAST_TRAN,EP_POSTCODE,EP_CITY,EP_COUNTRY,EP_SALES_DEPARTMENT,EP_SALES_TEAM,EP_FLEET_CATEGORY,EP_FLEET_TYPE
177619,87965366,Activé,Employee,NDXX,NaN,2001-11-24,2021-07-25,69006,LYON,FRANCE,Collaborateurs,Collaborateurs,Véhicules Légers,VL_PUR
177620,87277272,Activé,Standard,NaN,Action sociale sans hébergement pour personnes...,NaT,2021-06-01,59600,MAUBEUGE,FRANCE,Ventes Nationales,Ventes Nationales : Région Nord-Est,Véhicules Légers,VL_PUR
177621,87304937,Activé,Standard,4932,Transports de voyageurs par taxis,2018-09-25,2021-08-11,92130,ISSY LES MOULINEAUX,FRANCE,Ventes Locales,Ventes Locales : Région Nord,Véhicules Légers,VL_PUR
177622,87297555,Activé,Standard,NaN,Activités des sièges sociaux,NaT,NaT,92040,PARIS LA DEFENSE CEDEX,FRANCE,Ventes Nationales,Ventes Nationales : Grands Comptes,Véhicules Légers,VL_PUR
177623,87952484,Activé,Employee,9999,Joker,2001-11-24,2021-06-28,64150,MOURENX,FRANCE,Collaborateurs,Collaborateurs,Véhicules Légers,VL_PUR


## Nombre de modalités par variables

L'idée ici est d'analyser le nombre de valeurs uniques par variables et voir s'il est pertinent de garder les variables telles quelles. 

In [176]:
# Connaitre le nombre de valeurs uniques pour toutes les colonnes dans le but de savoir quelles variables convertir en categorielle

unique_counts = pd.DataFrame.from_records([(col, customer[col].nunique()) for col in customer.columns],
                          columns=['Nom_variables', 'Nombre_valeurs_uniques']).sort_values(by=['Nombre_valeurs_uniques'])

unique_counts

,Nom_variables,Nombre_valeurs_uniques
12,EP_FLEET_CATEGORY,2
1,EP_STATUS,3
10,EP_SALES_DEPARTMENT,3
2,EP_TYPE,5
13,EP_FLEET_TYPE,5
11,EP_SALES_TEAM,10
9,EP_COUNTRY,26
4,EP_NACE_NAME,590
3,EP_NACE_CODE,595
6,EP_DATE_LAST_TRAN,825


## Suppression de variables non pertinentes 

In [177]:
# Variables à supprimer pour non pertinences dans la suite de l'analyse

#to_drop = ["EP_NACE_CODE"] # Nous avons le nom des nace, donc leur code nous interesse pas pour l'analyse. 
#customer = customer.drop(columns = to_drop)
#customer.info()

## Données manquantes

Dans ce jeu de données, il y'a peu de données manquantes. La variable qui a le plus de données manquantes est *EP_NACE_NAME* representant 6.7% des données totales (11930). 

In [178]:
# Données manquantes

# Connaitre le nombre de valeurs uniques pour toutes les colonnes dans le but de savoir quelles variables convertir en categorielle

NA_counts = pd.DataFrame.from_records([(col
                                        , customer[col].isna().sum()
                                       , round((customer[col].isna().sum())/(customer[col].size),3)) for col in customer.columns],
                          columns=['Nom_variables', 'Nombre_valeurs_manquantes',"%NA"]).sort_values(by=['Nombre_valeurs_manquantes'], ascending=False)

NA_counts

,Nom_variables,Nombre_valeurs_manquantes,%NA
4,EP_NACE_NAME,11930,0.067
3,EP_NACE_CODE,7448,0.042
5,EP_DATE_ACTIVATION,7448,0.042
6,EP_DATE_LAST_TRAN,878,0.005
0,EP_CODE,0,0.000
1,EP_STATUS,0,0.000
2,EP_TYPE,0,0.000
7,EP_POSTCODE,0,0.000
8,EP_CITY,0,0.000
9,EP_COUNTRY,0,0.000


In [179]:
# Suppression des données manquantes
customer = customer.copy().dropna()

# Description sommaire des données de la table des clients

Le tableau ci dessous decrit brievement chaque variable. Il y'a dans ce tableau pour chque variable :
* count : le nombre d'observations (exclu les données manquantes :NAN)
* unique : le nombre de valeurs (modalités) discincts
* top : la valeur la plus fréquente
* freq : la frequence de la valeur la plus frequente (nombre brute=> comptage)
* freq_p : la proportion de la valeur la plus frequente ( en %)

Ainsi, on peut remarquer que plus de **99%** des clients ont un statut **actif** et sont en localisés en **France** dont la majorité sont de **Paris**. 

On remarque aussi que parmi les cinq types de flotte (VL_PUR, VL_MIXTE, PL_VIP, PL, PL_MIXTE), le type majoritairement detenu par les clients est **VL_PUR** (>70%), c'est à dire composée exclusivement par des vehicules légers. 

Par ailleurs, les departements de ventes sont majoritairement locaux (>64%) et l'equipe de ventes locale de la region du sud detient le plus de clients.

La date d'activation du statut du client le plus anciennement date de **février 1960** et la plus recente est du **29 juillet 2021**.

La date de dernière transaction la plus reculée est de **mai 2012** et la plus recente est du **11 août 2021**.

In [192]:
customer.describe(datetime_is_numeric=True)

,EP_CODE,EP_DATE_ACTIVATION,EP_DATE_LAST_TRAN
count,1.574010e+05,157401,157401
mean,8.731096e+07,2013-02-01 19:01:18.975355904,2021-07-30 01:05:45.062610944
min,8.700002e+07,1960-02-01 00:00:00,2012-05-09 00:00:00
25%,8.717868e+07,2007-12-06 00:00:00,2021-08-02 00:00:00
50%,8.728164e+07,2015-11-18 00:00:00,2021-08-09 00:00:00
75%,8.734274e+07,2019-04-11 00:00:00,2021-08-11 00:00:00
max,8.798020e+07,2021-07-29 00:00:00,2021-08-11 00:00:00
std,2.243607e+05,NaN,NaN


In [193]:
stats = customer.select_dtypes(include='object').describe()
stats

,EP_STATUS,EP_TYPE,EP_NACE_CODE,EP_NACE_NAME,EP_POSTCODE,EP_CITY,EP_COUNTRY,EP_SALES_DEPARTMENT,EP_SALES_TEAM,EP_FLEET_CATEGORY,EP_FLEET_TYPE
count,157401,157401,157401,157401,157401,157401,157401,157401,157401,157401,157401
unique,3,5,592,586,11628,17704,19,3,10,2,5
top,Activé,Standard,9999,Joker,75008,PARIS,FRANCE,Ventes Locales,Ventes Locales : Région Sud,Véhicules Légers,VL_PUR
freq,156424,134314,10467,10467,1534,8192,156928,108911,30022,149431,107782


In [194]:
# Ajout de la proportion en %

stats.loc["prop"] = pd.Series(stats.loc["freq"]/stats.loc["count"]).apply(lambda value : round(value,3)) # frequence en pourcentage
#stats = pd.DataFrame(np.insert(stats.values, stats.shape[0], freq_p, axis=0))
stats

,EP_STATUS,EP_TYPE,EP_NACE_CODE,EP_NACE_NAME,EP_POSTCODE,EP_CITY,EP_COUNTRY,EP_SALES_DEPARTMENT,EP_SALES_TEAM,EP_FLEET_CATEGORY,EP_FLEET_TYPE
count,157401,157401,157401,157401,157401,157401,157401,157401,157401,157401,157401
unique,3,5,592,586,11628,17704,19,3,10,2,5
top,Activé,Standard,9999,Joker,75008,PARIS,FRANCE,Ventes Locales,Ventes Locales : Région Sud,Véhicules Légers,VL_PUR
freq,156424,134314,10467,10467,1534,8192,156928,108911,30022,149431,107782
prop,0.994,0.853,0.066,0.066,0.01,0.052,0.997,0.692,0.191,0.949,0.685


# Autres analyses

In [183]:
customer.value_counts("EP_COUNTRY")

EP_COUNTRY
FRANCE         156928
MONACO            300
BELGIQUE           41
ALLEMAGNE          23
PAYS-BAS           21
LUXEMBOURG         20
SUISSE             17
ESPAGNE            16
ROYAUME-UNI        10
PORTUGAL            6
ITALIE              6
POLOGNE             3
HONGRIE             2
DANEMARK            2
AUTRICHE            2
MALTE               1
ANDORRE             1
FINLANDE            1
TURQUIE             1
dtype: int64

In [184]:
customer.value_counts("EP_COUNTRY",normalize=True)

EP_COUNTRY
FRANCE         0.996995
MONACO         0.001906
BELGIQUE       0.000260
ALLEMAGNE      0.000146
PAYS-BAS       0.000133
LUXEMBOURG     0.000127
SUISSE         0.000108
ESPAGNE        0.000102
ROYAUME-UNI    0.000064
PORTUGAL       0.000038
ITALIE         0.000038
POLOGNE        0.000019
HONGRIE        0.000013
DANEMARK       0.000013
AUTRICHE       0.000013
MALTE          0.000006
ANDORRE        0.000006
FINLANDE       0.000006
TURQUIE        0.000006
dtype: float64

In [185]:
customer.value_counts("EP_FLEET_CATEGORY")

EP_FLEET_CATEGORY
Véhicules Légers    149431
Poids Lourds          7970
dtype: int64

# Enregistrer les données propres

In [186]:
customer.to_csv("./data/table_customer_profile.csv",sep=';', encoding='latin-1',index=False)

# Clustering : kmode

In [187]:
# Kmodes : recherche du nbre de classe optimal

""" 
data = customer.iloc[:,1:]
cost = []
K = range(1,5)
for num_clusters in list(K):
    kmode = KModes(n_clusters=num_clusters, init = "random", n_init = 5, verbose=1)
    kmode.fit_predict(data)
    cost.append(kmode.cost_)

"""

' \ndata = customer.iloc[:,1:]\ncost = []\nK = range(1,5)\nfor num_clusters in list(K):\n    kmode = KModes(n_clusters=num_clusters, init = "random", n_init = 5, verbose=1)\n    kmode.fit_predict(data)\n    cost.append(kmode.cost_)\n\n'

In [188]:
"""

plt.plot(K, cost, 'bx-')
plt.xlabel('No. of clusters')
plt.ylabel('Cost')
plt.title('K optimal')
plt.show()

"""

"\n\nplt.plot(K, cost, 'bx-')\nplt.xlabel('No. of clusters')\nplt.ylabel('Cost')\nplt.title('K optimal')\nplt.show()\n\n"

In [189]:
#kmode = KModes(n_clusters=4, init = "random", n_init = 5, verbose=1)
#clusters = kmode.fit_predict(data)

In [190]:
#customer["cluster"] = clusters
#customer

In [191]:
# Encodage des données categorielles